In [6]:
!pip install flair
!pip install translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 6.7 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 788 kB 55.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.5 MB/s 
     |████████████████████████████████| 1.6 MB 55.0 MB/s 
     |████████████████████████████████| 19.7 MB 368 kB/s 
     |████████████████████████████████| 46 kB 5.1 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 4.2 MB 53.7 MB/s 
     |████████████████████████████████| 981 kB 56.5 MB/s 
     |████████████████████████████████| 199 kB 63.7 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 596 kB 69.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=0b61734455620f84

In [ ]:
#to run this code first run the below to lines
#pip install flair
#pip install translate

from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

import flair
from flair.data import Sentence
from flair.models import SequenceTagger
import requests
import json
from translate import Translator


def trans(word):                                        #function translates english word to telugu word
    translator= Translator(to_lang="telugu")
    translation = translator.translate(word)
    return translation

def te_stemmer(word):                                   # function will return a telugu word by trimming its ending
    
    if(word.endswith("ల") == True or
      word.endswith("ఓ") == True):
        return word[:len(word)-1]    
    
    if(word.endswith("డు") == True or
        word.endswith("ము") == True or
        word.endswith("వు") == True or
        word.endswith("లు") == True or
        word.endswith("ని") == True or
        word.endswith("ను") == True or
        word.endswith("చే") == True or
        word.endswith("తో") == True or
        word.endswith("కై") == True or
        word.endswith("లో") == True or
        word.endswith("కు") == True or
        word.endswith("కి") == True):
        return word[:len(word)-2]
    
    if(word.endswith("చేత") == True or
        word.endswith("తోడ") == True or
        word.endswith("వలన") == True or
        word.endswith("ఓరీ") == True or
        word.endswith("ఓయీ") == True or
        word.endswith("ఓసీ") == True):
        return word[:len(word)-3]
    
    if(word.endswith("లోపల") == True or
        word.endswith("కంటె") == True):
        return word[:len(word)-4]
    
    if(word.endswith("కొఱకు") == True or
        word.endswith("కొరకు") == True or
        word.endswith("పట్టి") == True or
        word.endswith("యొక్క") == True):
        return word[:len(word)-5]
    
    if(word.endswith("గూర్చి") == True):
        return word[:len(word)-6]
       
    if(word.endswith("గురించి") == True):
        return word[:len(word)-7]
    
    if(word.startswith("ఓ") == True):
        return word[1:]
    
    if(word.startswith("ఓయీ") == True or 
      word.startswith("ఓరీ") == True or
      word.startswith("ఓసీ") == True):
        return word[3:]
    
    return word

# This script to get the English to Telugu Translation
def GetTranslationFromIIITAPI(src):
    # response = requests.post("https://ssmt.iiit.ac.in/elil_iiith", json = {"text":src, "model":" "})
    data = {"text":src, "source_language":"eng", "target_language":"tel"}
    response = requests.post("https://ssmt.iiit.ac.in/mt_linker", data=json.dumps(data), headers={"Content-Type": "application/json",
    "Accept": "*/*"})
    model_translations = response.json()['data']
    return model_translations

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

f=open(r"train_en.txt",mode='r',encoding='utf-8')
text = f.read().splitlines()       #reading english dataset
t=open(r"train_te.txt",mode='r')
textt= t.read().splitlines()       #reading same telugu dataset
tl={}
for i in range(len(textt)):   
    for word in textt[i].split():         #removing extra characters from telugu words
         x = word.replace("\u200c","")
         x = word.replace(",","")
         x = word.replace(":","")
         x = word.replace(";","")
         x = word.replace("`","")
         tl[x] = '0'               #appending to dictionary          
my = [*range(1, len(text), 1)]
dic = list(zip(my,text,textt))  
df = pd.DataFrame(dic,columns=['ind','sent','tsent']) #creating a dataframe with 3 columns index, eng sentences, corresponding telugu sentences
d = {} 
tagger = SequenceTagger.load('ner')  #tagging PER, LOC, ORG, MISC
main_list=[]
for text in df['sent']:
  sentence = Sentence(text)
  tagger.predict(sentence)
  l=[]
  l.append(text)
  l1=[]
  for entity in sentence.get_spans('ner'):
    r = entity.text,entity.get_label("ner").value
    l1.append((entity.text,entity.get_label("ner").value))
  l.append(l1)
  main_list.append(l)                #sentence and its tagged words are appended
data = pd.DataFrame(main_list, columns=['text', 'annotation'])    #making dataframe
with open('new.txt' , 'w') as fr:     #file to store output   
      temp = []
      for annot in data.annotation: 
          if annot == []:
             temp.append([])
          else: 
             til = []  
             for i in range(len(annot)):         #tagging in IOB2 format
                 e_type = annot[i][1]
                 temp_list = annot[i][0].split() 
                 if len(temp_list) > 1:
                    til.append((temp_list[0],'B-' + e_type))
                    for w in temp_list[1:]:
                        til.append((w,'I-' + e_type))       
                 else:
                    til.append((temp_list[0],'B-' + e_type))
             temp.append(til)                        
      df['tags'] = temp   #adding new column to existing dataframe df('ind','sent','tsent')
      for i in range(df.shape[0]):
          if df.tags[i] == []:
             i = i + 1  #incrementing on getting a empty english tagged word set
          else:
             temp = df.tsent[i].split()  #splitting telugu sentence for performing stemming, translation and transliteration
             p = []
             for k in temp:
                p.append(te_stemmer(k))   #stemming
             for j in range(len(df.tags[i])):
                 x = GetTranslationFromIIITAPI(df.tags[i][j][0])    #transliteration
                 y = trans(df.tags[i][j][0])     #translation
                 if x in temp: 
                     tl[x] = df.tags[i][j][1]     #updating tl dictionary with tags
                 if y in temp:    
                     tl[y] = df.tags[i][j][1]                 
      for key in tl.keys():
          fr.writelines(key + ' ' + tl[key] +'\n')     #writing output to new.txt file
      fr.writelines('\n')

2022-06-11 14:04:38,130 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-11 14:04:40,452 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


RuntimeError: ignored

In [33]:
for i in range(998):
    print(data.annotation[i])

[]
[('Jehovahs', 'PER')]
[('India', 'LOC')]
[]
[]
[]
[]
[('V V Vinayak', 'PER')]
[('Hyderabad', 'LOC')]
[]
[('Rs', 'MISC')]
[]
[('Monaco-Ville', 'LOC'), ('French', 'MISC'), ('Mongasque', 'LOC')]
[('BBC', 'ORG')]
[]
[]
[]
[]
[]
[]
[('Shri Gangwar', 'PER'), ('Government of India', 'ORG'), ('Shri Narendra Modi', 'PER')]
[('Ministry of Science Technology Antiviral', 'ORG'), ('Ashutosh Sharma', 'PER'), ('PPEs', 'MISC'), ('Nano Mission', 'MISC'), ('Department of Science and Technology', 'ORG'), ('DST', 'ORG'), ('Ashwini Kumar Agrawal', 'PER'), ('Indian Institute of Technology', 'ORG'), ('Delhi', 'LOC')]
[]
[('Redmi 4', 'MISC')]
[('Parupalli Kashyap', 'PER'), ('Korea Open', 'MISC')]
[]
[('International Yoga Day', 'MISC'), ('Indian', 'MISC')]
[]
[]
[]
[]
[]
[]
[('Narendra Modi', 'PER')]
[]
[]
[]
[]
[('Israel', 'LOC')]
[]
[]
[]
[]
[]
[]
[]
[]
[('Constitution', 'ORG'), ('Bhupender Kumar Dutt', 'PER'), ('Constituent Assembly', 'ORG')]
[('Ganguly', 'PER')]
[]
[]
[]
[]
[('Delhi', 'LOC'), ('Rs', 'LO

In [ ]:

tl = [[],[],[('Monaco-Ville', 'LOC'), ('French', 'MISC'), ('Mongasque', 'LOC')]]
til = []  
temp = []
for annot in tl:
          if annot == []:
             temp.append([])
          else: 
             til = []  
             for i in range(len(annot)):         #tagging in IOB2 format
                 e_type = annot[i][1]
                 temp_list = annot[i][0].split() 
                 if len(temp_list) > 1:
                    til.append((temp_list[0],'B-' + e_type))
                    for w in temp_list[1:]:
                        til.append((w,'I-' + e_type))       
                 else:
                    til.append((temp_list[0],'B-' + e_type))
                 temp.append(til) 
print(temp)                                                 

In [36]:
for i in range(len(df.tags)):
    print(df.tags[i])

[]
[('Jehovahs', 'B-PER')]
[('India', 'B-LOC')]
[]
[]
[]
[]
[('V', 'B-PER'), ('V', 'I-PER'), ('Vinayak', 'I-PER')]
[('Hyderabad', 'B-LOC')]
[]
[('Rs', 'B-MISC')]
[]
[('Monaco-Ville', 'B-LOC'), ('French', 'B-MISC'), ('Mongasque', 'B-LOC')]
[('BBC', 'B-ORG')]
[]
[]
[]
[]
[]
[]
[('Shri', 'B-PER'), ('Gangwar', 'I-PER'), ('Government', 'B-ORG'), ('of', 'I-ORG'), ('India', 'I-ORG'), ('Shri', 'B-PER'), ('Narendra', 'I-PER'), ('Modi', 'I-PER')]
[('Ministry', 'B-ORG'), ('of', 'I-ORG'), ('Science', 'I-ORG'), ('Technology', 'I-ORG'), ('Antiviral', 'I-ORG'), ('Ashutosh', 'B-PER'), ('Sharma', 'I-PER'), ('PPEs', 'B-MISC'), ('Nano', 'B-MISC'), ('Mission', 'I-MISC'), ('Department', 'B-ORG'), ('of', 'I-ORG'), ('Science', 'I-ORG'), ('and', 'I-ORG'), ('Technology', 'I-ORG'), ('DST', 'B-ORG'), ('Ashwini', 'B-PER'), ('Kumar', 'I-PER'), ('Agrawal', 'I-PER'), ('Indian', 'B-ORG'), ('Institute', 'I-ORG'), ('of', 'I-ORG'), ('Technology', 'I-ORG'), ('Delhi', 'B-LOC')]
[]
[('Redmi', 'B-MISC'), ('4', 'I-MISC')]
[(

In [37]:
trans('Monaco-Ville')

'వి.'